In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder\
                    .master('local')\
                    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
                    .appName('spark-streaming-operation-write')\
                    .getOrCreate()

print(spark.version)

3.1.2


In [5]:
rawData = spark.readStream.format("kafka")\
                                .option("kafka.bootstrap.servers", "10.5.0.10:9092")\
                                .option("subscribe", "iot-data")\
                                .option("startingOffsets", "earliest")\
                                .load()

In [6]:
rawData

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [7]:
rawData.isStreaming

True

In [8]:
rawData.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
iotData = rawData.select("*")

In [10]:
iotData.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
query = rawData.writeStream.outputMode("append")\
                        .format("parquet")\
                        .option("path", "./tmp/iot-read-topic")\
                        .option("checkpointLocation", "./tmp/streaming-with-spark/consumer-checkpoint")\
                        .start()

In [13]:
query.recentProgress

[{'id': '4fda8c2a-0be5-4311-aabc-fe4f4abeded5',
  'runId': '10e49de4-ff1b-4e38-b076-d247d7478fd6',
  'name': None,
  'timestamp': '2021-09-30T13:23:38.712Z',
  'batchId': 0,
  'numInputRows': 1529,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 443.96051103368177,
  'durationMs': {'addBatch': 2567,
   'getBatch': 21,
   'latestOffset': 556,
   'queryPlanning': 230,
   'triggerExecution': 3443,
   'walCommit': 31},
  'stateOperators': [],
  'sources': [{'description': 'KafkaV2[Subscribe[iot-data]]',
    'startOffset': None,
    'endOffset': {'iot-data': {'0': 1529}},
    'numInputRows': 1529,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 443.96051103368177}],
  'sink': {'description': 'FileSink[./tmp/iot-read-topic]',
   'numOutputRows': -1}}]